<a href="https://colab.research.google.com/github/JuanesTc11/ST0245-001-/blob/master/CodigoEntrega3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import heapq
import gmplot
import pandas as pd

def main():
  data = pd.read_csv('calles_de_medellin_con_acoso.csv', sep=";")
  data = data.fillna({"harassmentRisk": data["harassmentRisk"].mean()})
  grafito=iniciar_grafito(data)
  print(camino_mas_corto_dijkstra(grafito,"(-75.5778046, 6.2029412)", "(-75.5762232, 6.266327)",1))
  print("distancia camino 1: "+str(contar_distancia(camino_mas_corto_dijkstra(grafito,"(-75.5778046, 6.2029412)", "(-75.5762232, 6.266327)",1))))
  print("acoso camino 1: "+str(contar_acoso(camino_mas_corto_dijkstra(grafito,"(-75.5778046, 6.2029412)", "(-75.5762232, 6.266327)",1))))
  print(camino_mas_corto_dijkstra(grafito,"(-75.5778046, 6.2029412)", "(-75.5762232, 6.266327)",2))
  print("distancia camino 2: "+str(contar_distancia(camino_mas_corto_dijkstra(grafito,"(-75.5778046, 6.2029412)", "(-75.5762232, 6.266327)",2))))
  print("acoso camino 2: "+str(contar_acoso(camino_mas_corto_dijkstra(grafito,"(-75.5778046, 6.2029412)", "(-75.5762232, 6.266327)",2))))
  print(camino_mas_corto_dijkstra(grafito,"(-75.5778046, 6.2029412)", "(-75.5762232, 6.266327)",3))
  print("distancia camino 3: "+str(contar_distancia(camino_mas_corto_dijkstra(grafito,"(-75.5778046, 6.2029412)", "(-75.5762232, 6.266327)",3))))
  print("acoso camino 3: "+str(contar_acoso(camino_mas_corto_dijkstra(grafito,"(-75.5778046, 6.2029412)", "(-75.5762232, 6.266327)",3))))
  graficar(camino_mas_corto_dijkstra(grafito,"(-75.5778046, 6.2029412)", "(-75.5762232, 6.266327)",1),camino_mas_corto_dijkstra(grafito,"(-75.5778046, 6.2029412)", "(-75.5762232, 6.266327)",2),camino_mas_corto_dijkstra(grafito,"(-75.5778046, 6.2029412)", "(-75.5762232, 6.266327)",3))

def leer_csv(dir):
  data = pd.read_csv(dir, sep=";")
  data = data.fillna({"harassmentRisk": data["harassmentRisk"].mean()})
  return data

def iniciar_grafito(data):
  grafito = {}
  grafito=crear_grafito(grafito, data["origin"].unique())
  grafito=crear_conexiones(grafito, data)
  return grafito

def crear_conexiones(grafito, data):
  for i in range(len(data)):
    if data.iloc[i]["oneway"] is True:
      grafito[data.iloc[i]["origin"]][data.iloc[i]["destination"]] = (data.iloc[i]["harassmentRisk"],data.iloc[i]["length"])
      grafito[data.iloc[i]["destination"]][data.iloc[i]["origin"]] = (data.iloc[i]["harassmentRisk"],data.iloc[i]["length"])
    else:
      grafito[data.iloc[i]["origin"]][data.iloc[i]["destination"]] = (data.iloc[i]["harassmentRisk"],data.iloc[i]["length"])
  return grafito

def crear_grafito(grafito, origenes):
  for i in origenes:
        Grafito = {}
        grafito[i] = Grafito
  return grafito

def llegamos(punto_origen, punto_destino, distancias, acoso_actual):
  arreglo = []
  padre = punto_destino
  arreglo.append(padre)
  while True:
    if padre is punto_origen:
      break;
    padre = distancias[padre][1]
    arreglo.append(padre)
  arreglo = list(reversed(arreglo))
  return arreglo

def camino_mas_corto_dijkstra(grafito, punto_origen, punto_destino,tipo):
    distancias = {vertice: [float('inf'), ""] for vertice in grafito}
    distancias[punto_origen][0] = 0
    distancias[punto_origen][1] = None
    return caminoscortos(distancias, grafito, punto_origen, punto_destino,tipo)

def caminoscortos(distancias, grafito, punto_origen, punto_destino, tipo):
  caminos_posibles = [(0, punto_origen)]
  while len(caminos_posibles) > 0:
      acoso_distancia_actual, vertice_actual = caminos_posibles.pop(0)
      acoso_distancia_actual
      if vertice_actual == punto_destino:
        return llegamos(punto_origen, punto_destino, distancias, acoso_distancia_actual)

      for Adyacente, acoso_distancia in grafito[vertice_actual].items():
        algoritmo = formula(acoso_distancia_actual, acoso_distancia, tipo)
        
        try:
          hacer_camino(algoritmo, acoso_distancia_actual, acoso_distancia, distancias, Adyacente, vertice_actual, caminos_posibles)
        except KeyError:
          continue

def formula(acoso_distancia_actual, acoso_distancia, tipo):
  acoso=acoso_distancia[0]
  distancia=acoso_distancia[1]
  if tipo==1:
    return acoso_distancia_actual+(acoso+distancia)
  elif tipo==2:
    return acoso_distancia_actual+((acoso**2)*(distancia**2))
  elif tipo==3:
    return acoso_distancia_actual+((acoso**2)+(distancia**2))

def hacer_camino(algoritmo, acoso_actual, acoso, distancias, Adyacente, vertice_actual, caminos_posibles):
    if algoritmo < distancias[Adyacente][0]:
      distancias[Adyacente][0] = algoritmo
      distancias[Adyacente][1] = vertice_actual
      heapq.heappush(caminos_posibles, (algoritmo, Adyacente))

def contar_distancia(camino):
  distancia_total=0
  for i in range(0,len(camino)-2):
    calle1=camino[i]
    calle2=camino[i+1]
    distancia_total = distancia_total+grafito[calle1][calle2][1]
  return distancia_total

def contar_acoso(camino):
  acoso_total=0
  for i in range(0,len(camino)-2):
    calle1=camino[i]
    calle2=camino[i+1]
    acoso_total = acoso_total+grafito[calle1][calle2][0]
  return acoso_total/len(camino)

def graficar(trayecto1,tryecto2,trayecto3):
  crearmapa(ordenarpuntos(trayecto1),ordenarpuntos(tryecto2),ordenarpuntos(trayecto3))

def ordenarpuntos(trayecto):
  camino = []
  for i in range(len(trayecto)-1):
    trayecto[i]=trayecto[i].replace("(","").replace(")","")
    y,x = trayecto[i].split(",")
    x=float(x)
    y=float(y)
    punto =(x, y)
    camino.append(punto)
  return camino

def crearmapa(trayecto1, trayecto2, trayecto3):
  puntosx_1, puntosy_1 = zip(*trayecto1)
  puntosx_2, puntosy_2 = zip(*trayecto2)
  puntosx_3, puntosy_3 = zip(*trayecto3)

  mapa = gmplot.GoogleMapPlotter(6.217, -75.567, 15)  
  mapa.marker(puntosx_1[0], puntosy_1[0], 'pink', title="origen")    
  mapa.marker(puntosx_1[-1], puntosy_1[-1], 'purple', title="Destino")  
  mapa.plot(puntosx_1, puntosy_1 , "blue", edge_width = 8.0)
  mapa.plot(puntosx_2, puntosy_2 , "red", edge_width = 8.0)
  mapa.plot(puntosx_3, puntosy_3 , "green", edge_width = 8.0)
  mapa.draw('camino.html')

main()

['(-75.5778046, 6.2029412)', '(-75.5778053, 6.2027845)', '(-75.5777932, 6.2026166)', '(-75.5778113, 6.2024225)', '(-75.5778475, 6.2022372)', '(-75.5778756, 6.2019281)', '(-75.5766166, 6.2014819)', '(-75.5764625, 6.2013993)', '(-75.5765714, 6.2012076)', '(-75.5773123, 6.2016662)', '(-75.5772161, 6.2024328)', '(-75.5771928, 6.202644)', '(-75.5771824, 6.2027283)', '(-75.5771701, 6.2028364)', '(-75.5771426, 6.2032701)', '(-75.5771278, 6.2034245)', '(-75.5771095, 6.2036164)', '(-75.5770744, 6.2039614)', '(-75.5770255, 6.2044966)', '(-75.5769312, 6.2055296)', '(-75.5769156, 6.205718)', '(-75.5768152, 6.2067883)', '(-75.576733, 6.2075852)', '(-75.5766584, 6.208379)', '(-75.576621, 6.208777)', '(-75.576537, 6.2096708)', '(-75.5765118, 6.2099386)', '(-75.5764963, 6.2101032)', '(-75.5762214, 6.212766)', '(-75.5762118, 6.2128694)', '(-75.5761366, 6.2136789)', '(-75.5761288, 6.2137628)', '(-75.5761017, 6.2140548)', '(-75.5760837, 6.2143035)', '(-75.5760354, 6.2147802)', '(-75.5760182, 6.2149508)',

In [ ]:
pip install gmplot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 164 kB 6.6 MB/s 
